In [38]:
!pip install langchain --quiet
!pip install openai --quiet
!pip install pdf2image --quiet
!pip install tabulate --quiet
!pip install tiktoken --quiet
!pip install unstructured --quiet

In [42]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("http://leavcom.com/pdf/DBpdf.pdf")

data = loader.load()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

You have 1 document(s) in your data
There are 13040 characters in your document


In [44]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)

print (f"You have {len(texts)} pages")

You have 8 pages


In [45]:
%%sql

DROP DATABASE IF EXISTS pdf_db;
CREATE DATABASE IF NOT EXISTS pdf_db;

In [46]:
%%sql

USE pdf_db;
DROP TABLE IF EXISTS pdf_docs1;
CREATE TABLE IF NOT EXISTS pdf_docs1 (
    id INT PRIMARY KEY,
    content TEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
    vector BLOB
);

In [47]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

In [48]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


In [49]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

In [50]:
for i, document in enumerate(texts):
    text_content = document.page_content

    embedding = embedder.embed_documents([text_content])[0]

    stmt = """
        INSERT INTO pdf_docs1 (
            id,
            content,
            vector
        )
        VALUES (
            %s,
            %s,
            JSON_ARRAY_PACK_F32(%s)
        )
    """

    db_connection.execute(stmt, (i+1, text_content, str(embedding)))

In [51]:
%%sql

USE pdf_db;
SELECT JSON_ARRAY_UNPACK_F32(vector)
FROM pdf_docs1
LIMIT 1;

JSON_ARRAY_UNPACK_F32(vector)


In [52]:
query_text = "Will object-oriented databases be commercially successful?"

query_embedding = embedder.embed_documents([query_text])[0]

stmt = """
    SELECT
        content,
        DOT_PRODUCT_F32(JSON_ARRAY_PACK_F32(%s), vector) AS score
    FROM pdf_docs1
    ORDER BY score DESC
    LIMIT 1
"""

results = db_connection.execute(stmt, str(query_embedding))

for row in results:
    print(row[0])

I N D U S T R Y T R E N D S

Whatever Happened to Object-Oriented Databases?

are gaining in popularity and are ex- pected to outsell even relational data- bases by 2003. And OO databases (see the “OO Database Orientation” sidebar) are still minor players with solid but strictly niche markets. Sales of relational databases have grown considerably faster than the sales of OO databases, and annual worldwide RDBMS revenues are now about 50 times larger.

Rick Cattell, distinguished engineer at Sun Microsystems, indicated, “Object- oriented databases are doing just ﬁne, and the news of their demise is highly exag- gerated. While their market [share] isn’t as big, they continue to be used in areas like CAD (computer-aided design) and telecommunications, where RDBMSs are not well suited.”

Neal Leavitt

A couple of years ago, industry

However, said Michael Stonebraker, chief technology ofﬁcer at Informix and an ORDBMS proponent and pioneer, “ODBMSs occupy a small niche market that has no br

In [53]:
import openai

prompt = f"The user asked: {query_text}. The most similar text from the document is: {row[0]}"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

Based on the information provided in the document, it seems that object-oriented databases have not become commercially successful in the overall database marketplace, and remain a niche technology with limited market share. Sales of relational databases continue to outstrip that of object-oriented databases, although there are certain specialized industries where object-oriented databases are still used. While the industry initially touted them as being on the rise and well-suited for the Internet age, this trend has not materialized at the scale that some expected.
